# Understanding the `geocode()` function

The <code>geocode()</code> method in the `arcgis.geocoding` module geocodes one location per request. The input address can be either a string containing a single line address, or divided among multiple parameters using a Python dict with keys that are named the same as those in the accepted address fields.

<img src="https://developers.arcgis.com/rest/geocode/api-reference/GUID-F2F78690-5FB5-4BF1-8633-26BF184C45A9-web.png"/>

The geocode() method supports finding the following types of
locations: 
* **Street addresses**
 * <code class="usertext">27488 Stanford Ave, Bowden, North Dakota</code>
 * <code class="usertext">380 New York St, Redlands, CA 92373</code>
* **Points of interest (POI) by name and type**
 * <code class="usertext">Disneyland</code>
 * <code class="usertext">banks in Paris</code>
 * <code class="usertext">los angeles starbucks</code>
 * <code class="usertext">mount everest</code>
* **Administrative place names** such as city, county, state, province, or country names
 * <code class="usertext">Seattle, Washington</code>
 * <code class="usertext">State of Mahārāshtra</code>
 * <code class="usertext">Liechtenstein</code>
 * <code class="usertext">mount everest</code>
* **Postal codes**
 * <code class="usertext">92591</code>
 * <code class="usertext">TW9 1DN</code>

There are several options for refining or restricting search results. These include:

* Specify output fields to return in the geocoding response with the outFields parameter.
* Specify the spatial reference of candidates with the outSR parameter.
* Limit the number of candidates with the maxLocations parameter.
* Confine the search results to a specified area with the searchExtent parameter.
* Use the location and distance parameters to prefer local candidates, which will then be returned higher in the candidates list.
* Filter search results using the category parameter.

Thus, in this guide we will cover:
 - [`geocode()` function signature and parameters](#geocode-function-signature-and-parameters)
   - [`address` parameter](#address-parameter)
     - [single line address](#single-line-address)
     - [multi field address](#multi-field-address)
   - [`search_extent` parameter](#search-extent-parameter)
   - [`location` parameter](#location-parameter)
   - [`distance` parameter](#distance-parameter)
   - [`category` parameter](#category-parameter)
   - [`out_sr` parameter](#out-sr-parameter)
   - [`out_fields` parameter](#out-fields-parameter)
   - [`max_locations` parameter](#max-locations-parameter)
   - [`for_storage` parameter](#for-storage-parameter)

<a id="geocode-function-signature-and-parameters"></a>
## geocode() function signature and parameters

The <code>geocode()</code> function supports searching for places and addresses in single field format, or in multifield format with the address components separated into mulitple parameters.

The code snippet below imports the geocode function and displays its signature and parameters along with a brief description:

In [1]:
from arcgis.geocoding import geocode

In [2]:
help(geocode)

Help on function geocode in module arcgis.geocoding._functions:

geocode(address, search_extent=None, location=None, distance=None, out_sr=None, category=None, out_fields='*', max_locations=20, magic_key=None, for_storage=False, geocoder=None)
    The geocode function geocodes one location per request.
    
    Inputs:
    
       address - Specifies the location to be geocoded. This can be a string
       containing the street address, place name, postal code, or POI.
    
        Alternatively, this can be a dictionary containing the various address fields accepted
        by the corresponding geocoder. These fields are listed in the addressFields property of
        the associated geocoder. For example, if the address_fields of a geocoder includes fields
        with the following names: Street, City, State and Zone, then the
        address argument is of the form:
        {
          Street: "1234 W Main St",
          City: "Small Town",
          State: "WA",
          Zone: "99

<a id="address-parameter"></a>
### address parameter
<a id="single-line-address"></a>
#### single line address
The <b>address</b> parameter specifies the location to be geocoded. This can be a string containing the single line address, i.e street address, place name, postal code, or POI.

#### Example: plot an address using a single line address

In [3]:
from arcgis.gis import GIS

gis = GIS()
map = gis.map("Redlands, CA")
map

![single line geocoding](http://esri.github.io/arcgis-python-api/notebooks/nbimages/guide_tools_geocoding_01.png)

In [4]:
single_line_address = "380 New York Street, Redlands, CA 92373"

In [5]:
# geocode the single line address and plot the location of the first geocode result on the map
esrihq = geocode(single_line_address)[0]

# add a popup to the matched location
popup = { 
    "title" : "Esri Headquarters", 
    "content" : esrihq['address']
    }
map.draw(esrihq['location'], popup)

<a id="multi-field-address"></a>
#### multi field address
Alternatively, the address can be specified in a multifield format using a dictionary containing the various address fields accepted by the corresponding geocode service. 

In order to provide a way to find addresses in many different countries, which may use different addressing formats, the <code>geocode()</code> method uses standardized field names for submitting address components. 

The Geocoder's 'addressFields' property specifies the various address fields accepted by it when geocoding addresses. The neighborhood, city, subregion, and region parameters represent typical administrative divisions within a country. They may have different contexts for different countries, and not all administrative divisions are used in all countries. For instance, with addresses in the United States, only the city (city) and region(state) parameters are used; for addresses in Mexico, the neighborhood parameter is used for districts (colonias) within a city, city for municipalities (municipios), and the region parameter for states (estados); Spain uses all four administrative divisions.

For example, if the addressFields of a geocode service resource includes fields with the following names: Address, City, Region and Postal, then the address argument is of the form below.

#### Example: plot an address using a multiple field address

In [6]:
multi_field_address = { 
    "Address" : "380 New York Street",
    "City" : "Redlands",
    "Region" : "CA",
    "Postal" : 92373
    }

In [7]:
map = gis.map("Redlands, CA")
map

![sample output](http://esri.github.io/arcgis-python-api/notebooks/nbimages/guide_tools_geocoding_01.png)

In [8]:
# geocode the multi field address and plot the location of the first geocode result on the map

esrihq = geocode(multi_field_address)[0]
popup = { 
    "title" : "Esri Headquarters", 
    "content" : esrihq['address']
    }
map.draw(esrihq['location'], popup)

The `geocode()` method retuns an array of potential address matches (also refered to as address candidates). Each address candidate is represented as a Python dictionary with the following keys:

In [9]:
esrihq.keys()

dict_keys(['extent', 'score', 'location', 'attributes', 'address'])

The dict keys-values are the following:
* <code>score</code> represents the level of confidence of the geocoder in the match, ranked from 0-100
* <code>location</code> contains the (x, y) location of the match,
* <code>address</code> includes the matched address,
* <code>attributes</code> include several parameters from the 'Candidate Fields' property above, and
* <code>extent</code> specifies an appropriate extent for the matched address.

<a id="search-extent-parameter"></a>
### search_extent parameter

A set of bounding box coordinates that limit the search area to a specific region. This is especially useful for applications in which a user will search for places and addresses within the current map extent.

You can specify the spatial reference of the searchExtent coordinates, which is necessary if the map spatial reference is different than that of the geocoding service; otherwise, the spatial reference of the coordinates is assumed to be the same as that of the geocoding service.

The input can either be a comma-separated list of coordinates defining the bounding box or a JSON envelope object. The spatial reference of the bounding box coordinates can be included if an envelope object is used.

#### Example: Starbucks around Union Square in San Francisco, CA
The example below finds and plots upto 100 Starbucks locations around Union Square in San Francisco, CA. The extent parameter is obtained from the geocoder's geocoding result for Union Square (<code>unionsquare['extent']</code>) and passed into the next geocode() request for 100 Starbucks locations:

In [10]:
unionsquare = geocode("Union Square, San Francisco, CA")[0]
map = gis.map(unionsquare)
map

![sample output](http://esri.github.io/arcgis-python-api/notebooks/nbimages/guide_tools_geocoding_02.png)

In [11]:
# find and plot upto 100 Starbucks(TM) locations around Union Square in San Francisco, CA
starbucks = geocode("Starbucks", unionsquare['extent'], max_locations=100)
for starbuck in starbucks:
    map.draw(starbuck['location'])

<a id="location-parameter"></a>
### location parameter

Defines an origin point location that is used with the distance parameter to sort geocoding candidates based upon their proximity to the location. The distance parameter specifies the radial distance from the location in meters. The priority of candidates within this radius is boosted relative to those outside the radius.

This is useful in mobile applications where a user will want to search for places in the vicinity of their current GPS location; the location and distance parameters can be used in this scenario.

The location parameter can be specified without specifying a distance. If distance is not specified, it defaults to 50000 meters.

The location can be represented with a simple comma-separated syntax (x,y), or as a JSON point object. If the comma-separated syntax is used, the spatial reference of the coordinates must be WGS84. Otherwise, the spatial reference of the point coordinates can be defined in the JSON object.

Example using simple syntax (WGS84):
location=-117.196,34.056
JSON example with a spatial reference:
<code>
location=
{
    "x": -13046165.572,
    "y": 4036389.847,
    "spatialReference": {
        "wkid": 102100
    }
}
</code>

<a id="distance-parameter"></a>
### distance parameter

Specifies the radius of an area around a point location that is used to boost the rank of geocoding candidates so that candidates closest to the location are returned first. The distance value is in meters.

If the distance parameter is specified, then the location parameter must be specified as well.

It is important to note that unlike the searchExtent parameter, the location and distance parameters allow searches to extend beyond the specified search radius. They are not used to filter results, but rather to rank resulting candidates based on their distance from a location. You must pass a searchExtent value in addition to location and distance if you want to confine the search results to a specific area.

Example of searching within two miles of the current extent:
<code>distance=3218.69</code>

<a id="category-parameter"></a>
### category parameter

A place or address type which can be used to filter geocoding results. The parameter supports input of single category values or multiple comma-separated values. The category parameter can be passed in a request with or without a single line address input. 

Example of category filtering with a single category:

<code>category="Address"</code>

Example of category filtering with multiple categories:

<code>category="Address,Postal"</code>

Note:
The category parameter is only functional when used with single line address input. It does not work with multi field addresses; specifically the address, neighborhood, city, region, subregion, countryCode, and postal parameters.

### Example: Indian Food around Union Square in San Francisco, CA

In [12]:
unionsquare = geocode("Union Square, San Francisco, CA")[0]

map = gis.map(unionsquare)
map

![sample output](http://esri.github.io/arcgis-python-api/notebooks/nbimages/guide_tools_geocoding_03.png)

In [13]:
# find and plot upto 100 Indian restaurants around Union Square in San Francisco, CA

restaurants = geocode(None, unionsquare['extent'], category="Indian Food", max_locations=100)
for restaurant in restaurants:
    popup = { 
    "title" : restaurant['address'], 
    "content" : "Phone: " + restaurant['attributes']['Phone']
    }
    map.draw(restaurant['location'], popup)

### Example: Searching for multiple categories and plotting them with different smbols
In the example below, we search for Indian food as well as Thai Food in San Francisco and plot their locations using different symbols based on the 'Type' attribute:

In [14]:
categories = "Indian Food, Thai Food"

In [15]:
unionsquare = geocode("Union Square, San Francisco, CA")[0]

map = gis.map(unionsquare)
map

![sample output](http://esri.github.io/arcgis-python-api/notebooks/nbimages/guide_tools_geocoding_04.png)

In [16]:
# find and plot upto 100 Indian and Thai restaurants in San Francisco, CA

thaisymbol = {
"type": "esriSMS",
 "style": "esriSMSSquare",
 "color": [76,115,0,255],
 "size": 8,
 "angle": 0,
 "xoffset": 0,
 "yoffset": 0,
 "outline": 
  {
  "color": [152,230,0,255],
   "width": 1
  }
}

restaurants = geocode(None, unionsquare['extent'], category=categories, max_locations=100)
for restaurant in restaurants:
    popup = { 
    "title" : restaurant['address'], 
    "content" : "Phone: " + restaurant['attributes']['Phone']
        }
    if restaurant['attributes']['Type'] == 'Thai Food':
        map.draw(restaurant['location'], popup, thaisymbol) # use a green square symbol for Thai food
    else:
        map.draw(restaurant['location'], popup)

<a id="out-sr-parameter"></a>
### out_sr  parameter

The spatial reference of the x/y coordinates returned by the geocode method. This is useful for applications using a map with a spatial reference different than that of the geocoder.

The spatial reference can be specified as either a well-known ID (WKID) or as a <a href="http://resources.arcgis.com/EN/HELP/REST/APIREF/GEOMETRY.HTML#SR">JSON spatial reference object</a>. If outSR is not specified, the spatial reference of the output locations is the same as that of the geocoder. The World Geocoding Service spatial reference is WGS84 (WKID = 4326).

For a list of valid WKID values, see <a href="http://resources.arcgis.com/EN/HELP/REST/APIREF/PCS.HTML">Projected Coordinate Systems</a> and <a href="http://resources.arcgis.com/EN/HELP/REST/APIREF/GCS.HTML">Geographic Coordinate Systems</a>.

Example (102100 is the WKID for the Web Mercator projection):

<code>out_sr=102100</code>

<a id="out-fields-parameter"></a>
### out_fields parameter

The list of fields to be returned in the response. Descriptions for each of these fields are available in the <a href="/rest/geocode/api-reference/geocoding-service-output.htm#ESRI_SECTION1_42D7D3D0231241E9B656C01438209440">Output fields section of this document</a>.

The returned address, x/y coordinates of the match location, match score, spatial reference, extent of the output feature, and the addr_type (match level) are returned by default.

Example that returns all output fields:

<code>out_fields=*</code>

Example that returns the specified fields only:

<code>out_fields="AddrNum,StName,City"</code>

<a id="max-locations-parameter"></a>
### max_locations parameter

The maximum number of locations to be returned by a search, up to the maximum number allowed by the geocoder. If not specified, then all matching candidates up to the maximum are returned.

The World Geocoding Service allows up to 20 candidates to be returned for a single request. Note that up to 50 POI candidates can be returned.

Example:

<code>max_locations=10</code>

<a id="for-storage-parameter"></a>
### for_storage parameter

Specifies whether the results of the operation will be persisted. The default value is false, which indicates the results of the operation can't be stored, but they can be temporarily displayed on a map for instance. If you store the results, in a database for example, you need to set this parameter to true.

Applications are contractually prohibited from storing the results of geocoding transactions unless they make the request by passing the forStorage parameter with a value of true.

ArcGIS Online service credits are deducted from the organization account for each geocode transaction that includes the forStorage parameter with a value of true. Refer to the <a href="http://www.esri.com/SOFTWARE/ARCGIS/ARCGISONLINE/CREDITS">ArcGIS Online service credits overview page</a> for more information on how credits are charged.

To learn more about free and paid geocoding operations, see <a href="/rest/geocode/api-reference/geocoding-free-vs-paid.htm">this help topic</a>.

Example:

<code>for_storage=True</code>